# LDOS Path Analysis

This notebook provides interactive analysis of end-to-end callback chains and timing paths.

## Contents
1. Setup and Configuration
2. Load Trace Data
3. Callback Chain Analysis
4. Path Timeline Visualization
5. Latency Breakdown
6. Bottleneck Identification
7. Cross-Scenario Comparison
8. Export Results

## 1. Setup and Configuration

In [ ]:
# Standard imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import seaborn as sns
from pathlib import Path
import json
import sys
import yaml
import warnings

# Configure
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette('husl')
plt.rcParams['figure.figsize'] = (14, 8)
warnings.filterwarnings('ignore')

# Project paths
WS_ROOT = Path('..').resolve()
TRACES_DIR = WS_ROOT / 'traces'
RESULTS_DIR = WS_ROOT / 'results'
CONFIGS_DIR = WS_ROOT / 'configs'
ANALYSIS_DIR = WS_ROOT / 'analysis'
OUTPUT_DIR = ANALYSIS_DIR / 'output' / 'paths'

print(f"Workspace: {WS_ROOT}")
print(f"Traces directory: {TRACES_DIR}")
print(f"Output directory: {OUTPUT_DIR}")

In [ ]:
# Add analysis directory to path for imports
sys.path.insert(0, str(ANALYSIS_DIR))

# Try importing e2e_path_analyzer
try:
    from e2e_path_analyzer import E2EPathAnalyzer, CallbackInfo, CallbackChain
    HAS_ANALYZER = True
    print("E2E Path Analyzer loaded successfully")
except ImportError as e:
    HAS_ANALYZER = False
    print(f"E2E Path Analyzer not available: {e}")
    print("Some features will be limited")

In [ ]:
# Load objectives configuration
objectives_path = CONFIGS_DIR / 'objectives.yaml'

if objectives_path.exists():
    with open(objectives_path) as f:
        objectives = yaml.safe_load(f)
    print("Loaded objectives.yaml")
    print(f"Defined paths: {list(objectives.get('paths', {}).keys())}")
else:
    objectives = {}
    print("objectives.yaml not found. Using defaults.")

## 2. Load Trace Data

Load callback timing data from experiment results.

In [ ]:
def find_trace_directories(base_dir: Path) -> dict:
    """
    Find all trace directories organized by scenario.
    """
    trace_dirs = {}
    
    if not base_dir.exists():
        return trace_dirs
    
    # Check both traces/ and results/ directories
    for search_dir in [base_dir, base_dir.parent / 'results']:
        if not search_dir.exists():
            continue
            
        for scenario_dir in search_dir.iterdir():
            if not scenario_dir.is_dir():
                continue
            
            # Look for LTTng trace directories
            lttng_dirs = list(scenario_dir.glob('**/metadata'))
            if lttng_dirs:
                trace_dirs[scenario_dir.name] = [d.parent for d in lttng_dirs]
            
            # Also check for processed callback data
            callback_files = list(scenario_dir.glob('*callback*.json')) + \
                            list(scenario_dir.glob('*callback*.csv'))
            if callback_files:
                if scenario_dir.name not in trace_dirs:
                    trace_dirs[scenario_dir.name] = []
                trace_dirs[scenario_dir.name].extend(callback_files)
    
    return trace_dirs

trace_dirs = find_trace_directories(TRACES_DIR)
print(f"Found traces for scenarios: {list(trace_dirs.keys())}")

for scenario, dirs in trace_dirs.items():
    print(f"  {scenario}: {len(dirs)} trace(s)")

In [ ]:
def load_callback_data_from_results(results_dir: Path) -> pd.DataFrame:
    """
    Load callback timing data from result JSON files.
    """
    all_rows = []
    
    if not results_dir.exists():
        return pd.DataFrame()
    
    for scenario_dir in results_dir.iterdir():
        if not scenario_dir.is_dir():
            continue
        
        scenario = scenario_dir.name
        
        for json_file in scenario_dir.glob('*_result.json'):
            try:
                with open(json_file) as f:
                    data = json.load(f)
                
                # Extract callback timing if available
                if 'callbacks' in data:
                    for cb in data['callbacks']:
                        cb['scenario'] = scenario
                        cb['trial'] = json_file.stem
                        all_rows.append(cb)
                elif 'timing' in data:
                    # Alternative format with timing breakdown
                    for phase, timing in data['timing'].items():
                        all_rows.append({
                            'scenario': scenario,
                            'trial': json_file.stem,
                            'callback': phase,
                            'duration_ms': timing.get('duration_ms', 0),
                            'start_ns': timing.get('start_ns', 0),
                            'end_ns': timing.get('end_ns', 0)
                        })
            except Exception as e:
                pass
    
    return pd.DataFrame(all_rows) if all_rows else pd.DataFrame()

# Try loading callback data
callback_df = load_callback_data_from_results(RESULTS_DIR)
print(f"Loaded {len(callback_df)} callback records")

if len(callback_df) > 0:
    display(callback_df.head())

In [ ]:
# Create synthetic callback data for demonstration if no real data
def create_demo_callback_data() -> pd.DataFrame:
    """
    Create synthetic callback chain data for demonstration.
    """
    np.random.seed(42)
    
    # Define typical ROS 2 callback chains for manipulation
    chains = {
        'planning': [
            ('goal_callback', 5, 15),
            ('compute_ik', 50, 150),
            ('plan_path', 200, 800),
            ('validate_trajectory', 20, 60),
        ],
        'control': [
            ('joint_state_callback', 0.5, 1.5),
            ('compute_command', 0.8, 2.0),
            ('publish_command', 0.2, 0.5),
        ],
        'perception': [
            ('image_callback', 10, 30),
            ('process_pointcloud', 30, 80),
            ('update_scene', 15, 40),
        ]
    }
    
    rows = []
    scenarios = ['baseline', 'cpu_load', 'msg_load']
    load_factors = {'baseline': 1.0, 'cpu_load': 1.8, 'msg_load': 1.3}
    
    for scenario in scenarios:
        factor = load_factors[scenario]
        
        for trial_id in range(30):
            for chain_name, callbacks in chains.items():
                current_time = 0
                
                for cb_name, min_ms, max_ms in callbacks:
                    # Add some noise and load factor
                    duration = np.random.uniform(min_ms, max_ms) * factor
                    duration *= np.random.uniform(0.9, 1.1)  # Additional noise
                    
                    rows.append({
                        'scenario': scenario,
                        'trial': f'trial_{trial_id:03d}',
                        'chain': chain_name,
                        'callback': cb_name,
                        'start_ms': current_time,
                        'duration_ms': duration,
                        'end_ms': current_time + duration
                    })
                    
                    current_time += duration
    
    return pd.DataFrame(rows)

# Use demo data if no real data available
if len(callback_df) == 0:
    print("\nNo callback data found. Using demo data for visualization.")
    callback_df = create_demo_callback_data()
    print(f"Created {len(callback_df)} demo callback records")
    
    display(callback_df.head(10))

## 3. Callback Chain Analysis

In [ ]:
# Analyze callback chains
if len(callback_df) > 0 and 'chain' in callback_df.columns:
    print("=== Callback Chain Summary ===")
    
    chain_summary = callback_df.groupby(['scenario', 'chain']).agg({
        'duration_ms': ['mean', 'std', 'min', 'max', lambda x: x.quantile(0.95)],
        'trial': 'nunique'
    }).round(2)
    
    chain_summary.columns = ['mean_ms', 'std_ms', 'min_ms', 'max_ms', 'p95_ms', 'n_trials']
    display(chain_summary)

In [ ]:
# Per-callback analysis
if len(callback_df) > 0:
    print("\n=== Per-Callback Statistics ===")
    
    cb_summary = callback_df.groupby(['scenario', 'callback']).agg({
        'duration_ms': ['mean', 'std', lambda x: x.quantile(0.95)]
    }).round(2)
    
    cb_summary.columns = ['mean_ms', 'std_ms', 'p95_ms']
    
    # Reshape for comparison
    cb_pivot = cb_summary.reset_index().pivot(
        index='callback', 
        columns='scenario', 
        values='mean_ms'
    )
    
    display(cb_pivot)

## 4. Path Timeline Visualization

In [ ]:
def plot_callback_timeline(df: pd.DataFrame, scenario: str, trial: str = None, chain: str = None):
    """
    Plot a Gantt-style timeline of callback executions.
    """
    # Filter data
    plot_df = df[df['scenario'] == scenario].copy()
    
    if trial:
        plot_df = plot_df[plot_df['trial'] == trial]
    else:
        # Use first trial
        trial = plot_df['trial'].iloc[0]
        plot_df = plot_df[plot_df['trial'] == trial]
    
    if chain:
        plot_df = plot_df[plot_df['chain'] == chain]
    
    if len(plot_df) == 0:
        print(f"No data for scenario={scenario}, trial={trial}, chain={chain}")
        return
    
    # Create figure
    fig, ax = plt.subplots(figsize=(14, max(6, len(plot_df) * 0.4)))
    
    # Color by chain
    if 'chain' in plot_df.columns:
        chains = plot_df['chain'].unique()
        colors = dict(zip(chains, sns.color_palette('husl', len(chains))))
    else:
        colors = {'default': 'steelblue'}
    
    # Plot bars
    y_positions = range(len(plot_df))
    
    for idx, (_, row) in enumerate(plot_df.iterrows()):
        chain_name = row.get('chain', 'default')
        color = colors.get(chain_name, 'gray')
        
        ax.barh(idx, row['duration_ms'], left=row.get('start_ms', 0),
                height=0.6, color=color, alpha=0.8, edgecolor='black', linewidth=0.5)
        
        # Add duration label
        ax.text(row.get('start_ms', 0) + row['duration_ms'] / 2, idx,
                f"{row['duration_ms']:.1f}ms", ha='center', va='center', fontsize=8)
    
    # Labels
    ax.set_yticks(y_positions)
    ax.set_yticklabels(plot_df['callback'])
    ax.set_xlabel('Time (ms)')
    ax.set_ylabel('Callback')
    ax.set_title(f'Callback Timeline - {scenario} ({trial})')
    
    # Legend
    if 'chain' in plot_df.columns:
        patches = [mpatches.Patch(color=c, label=name) for name, c in colors.items()]
        ax.legend(handles=patches, loc='upper right')
    
    ax.invert_yaxis()
    plt.tight_layout()
    plt.show()

if len(callback_df) > 0:
    scenarios = callback_df['scenario'].unique()
    print(f"Available scenarios: {scenarios}")
    
    # Plot timeline for first scenario
    plot_callback_timeline(callback_df, scenarios[0])

In [ ]:
# Compare timelines across scenarios
if len(callback_df) > 0 and 'chain' in callback_df.columns:
    fig, axes = plt.subplots(1, len(callback_df['scenario'].unique()), 
                             figsize=(6*len(callback_df['scenario'].unique()), 8),
                             sharey=True)
    
    if len(callback_df['scenario'].unique()) == 1:
        axes = [axes]
    
    # Get average timings per callback
    avg_timings = callback_df.groupby(['scenario', 'chain', 'callback']).agg({
        'duration_ms': 'mean',
        'start_ms': 'mean'
    }).reset_index()
    
    for ax, scenario in zip(axes, callback_df['scenario'].unique()):
        scenario_df = avg_timings[avg_timings['scenario'] == scenario]
        
        chains = scenario_df['chain'].unique()
        colors = dict(zip(chains, sns.color_palette('husl', len(chains))))
        
        y_positions = range(len(scenario_df))
        
        for idx, (_, row) in enumerate(scenario_df.iterrows()):
            color = colors.get(row['chain'], 'gray')
            ax.barh(idx, row['duration_ms'], left=row['start_ms'],
                    height=0.6, color=color, alpha=0.8, edgecolor='black', linewidth=0.5)
        
        ax.set_yticks(y_positions)
        ax.set_yticklabels(scenario_df['callback'])
        ax.set_xlabel('Time (ms)')
        ax.set_title(f'{scenario}')
        ax.invert_yaxis()
    
    plt.suptitle('Average Callback Timelines by Scenario', y=1.02)
    plt.tight_layout()
    plt.show()

## 5. Latency Breakdown

In [ ]:
# Compute latency breakdown per chain
if len(callback_df) > 0 and 'chain' in callback_df.columns:
    print("=== Latency Breakdown by Chain ===")
    
    # Sum callback durations per trial and chain
    chain_totals = callback_df.groupby(['scenario', 'trial', 'chain'])['duration_ms'].sum().reset_index()
    
    # Summary statistics
    chain_stats = chain_totals.groupby(['scenario', 'chain'])['duration_ms'].agg([
        'mean', 'std', 'min', 'max'
    ]).round(2)
    
    display(chain_stats)
    
    # Stacked bar chart
    fig, ax = plt.subplots(figsize=(12, 6))
    
    scenarios = callback_df['scenario'].unique()
    chains = callback_df['chain'].unique()
    
    # Compute mean per scenario/chain
    pivot = chain_totals.groupby(['scenario', 'chain'])['duration_ms'].mean().unstack(fill_value=0)
    
    pivot.plot(kind='bar', stacked=True, ax=ax, colormap='husl')
    
    ax.set_xlabel('Scenario')
    ax.set_ylabel('Total Latency (ms)')
    ax.set_title('Latency Breakdown by Callback Chain')
    ax.legend(title='Chain', bbox_to_anchor=(1.02, 1), loc='upper left')
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Percentage breakdown
if len(callback_df) > 0 and 'chain' in callback_df.columns:
    print("\n=== Percentage Breakdown ===")
    
    # Mean callback durations
    mean_durations = callback_df.groupby(['scenario', 'callback'])['duration_ms'].mean()
    
    for scenario in callback_df['scenario'].unique():
        scenario_total = mean_durations[scenario].sum()
        print(f"\n{scenario} (total: {scenario_total:.1f}ms):")
        
        for callback, duration in mean_durations[scenario].sort_values(ascending=False).items():
            pct = duration / scenario_total * 100
            print(f"  {callback}: {duration:.1f}ms ({pct:.1f}%)")

## 6. Bottleneck Identification

In [ ]:
def identify_bottlenecks(df: pd.DataFrame, threshold_pct: float = 20.0):
    """
    Identify callbacks that consume more than threshold_pct of total latency.
    """
    bottlenecks = []
    
    for scenario in df['scenario'].unique():
        scenario_df = df[df['scenario'] == scenario]
        
        # Mean duration per callback
        mean_durations = scenario_df.groupby('callback')['duration_ms'].mean()
        total = mean_durations.sum()
        
        for callback, duration in mean_durations.items():
            pct = duration / total * 100
            if pct >= threshold_pct:
                bottlenecks.append({
                    'scenario': scenario,
                    'callback': callback,
                    'mean_ms': duration,
                    'pct_total': pct,
                    'is_bottleneck': True
                })
    
    return pd.DataFrame(bottlenecks)

if len(callback_df) > 0:
    print("=== Bottleneck Analysis (>20% of total latency) ===")
    
    bottleneck_df = identify_bottlenecks(callback_df, threshold_pct=20.0)
    
    if len(bottleneck_df) > 0:
        display(bottleneck_df.sort_values('pct_total', ascending=False))
    else:
        print("No callbacks exceed 20% of total latency.")

In [ ]:
# Heatmap of callback contributions
if len(callback_df) > 0:
    # Compute percentage contribution
    pct_contribution = callback_df.groupby(['scenario', 'callback'])['duration_ms'].mean().unstack(fill_value=0)
    pct_contribution = pct_contribution.div(pct_contribution.sum(axis=1), axis=0) * 100
    
    fig, ax = plt.subplots(figsize=(14, 6))
    
    sns.heatmap(pct_contribution, annot=True, fmt='.1f', cmap='YlOrRd',
                cbar_kws={'label': '% of Total Latency'}, ax=ax)
    
    ax.set_title('Callback Contribution to Total Latency (%)')
    ax.set_xlabel('Callback')
    ax.set_ylabel('Scenario')
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Identify load-sensitive callbacks
if len(callback_df) > 0 and 'baseline' in callback_df['scenario'].unique():
    print("\n=== Load-Sensitive Callbacks ===")
    print("(Callbacks with largest increase under load)\n")
    
    # Mean durations
    mean_by_scenario = callback_df.groupby(['scenario', 'callback'])['duration_ms'].mean().unstack(level=0)
    
    if 'baseline' in mean_by_scenario.columns:
        for load_scenario in [s for s in mean_by_scenario.columns if s != 'baseline']:
            pct_change = ((mean_by_scenario[load_scenario] - mean_by_scenario['baseline']) / 
                         mean_by_scenario['baseline'] * 100)
            
            print(f"\n{load_scenario} vs baseline:")
            for callback, change in pct_change.sort_values(ascending=False).head(5).items():
                baseline_val = mean_by_scenario.loc[callback, 'baseline']
                load_val = mean_by_scenario.loc[callback, load_scenario]
                print(f"  {callback}: {baseline_val:.1f}ms -> {load_val:.1f}ms ({change:+.1f}%)")

## 7. Cross-Scenario Comparison

In [ ]:
# Box plots per callback across scenarios
if len(callback_df) > 0:
    # Select top callbacks by mean duration
    top_callbacks = callback_df.groupby('callback')['duration_ms'].mean().nlargest(6).index.tolist()
    
    fig, axes = plt.subplots(2, 3, figsize=(15, 10))
    axes = axes.flatten()
    
    for ax, callback in zip(axes, top_callbacks):
        data = callback_df[callback_df['callback'] == callback]
        
        sns.boxplot(data=data, x='scenario', y='duration_ms', ax=ax)
        ax.set_title(f'{callback}')
        ax.set_xlabel('')
        ax.set_ylabel('Duration (ms)')
    
    plt.suptitle('Callback Duration Distribution Across Scenarios', y=1.02)
    plt.tight_layout()
    plt.show()

In [ ]:
# Chain-level comparison
if len(callback_df) > 0 and 'chain' in callback_df.columns:
    # Total chain duration per trial
    chain_durations = callback_df.groupby(['scenario', 'trial', 'chain'])['duration_ms'].sum().reset_index()
    
    fig, ax = plt.subplots(figsize=(12, 6))
    
    sns.boxplot(data=chain_durations, x='chain', y='duration_ms', hue='scenario', ax=ax)
    
    ax.set_xlabel('Callback Chain')
    ax.set_ylabel('Total Duration (ms)')
    ax.set_title('Chain Duration Distribution by Scenario')
    ax.legend(title='Scenario')
    
    plt.tight_layout()
    plt.show()

## 8. Export Results

In [ ]:
def export_path_analysis(callback_df: pd.DataFrame, bottleneck_df: pd.DataFrame, 
                         output_dir: Path):
    """
    Export path analysis results.
    """
    output_dir.mkdir(parents=True, exist_ok=True)
    
    # Callback statistics
    if len(callback_df) > 0:
        cb_stats = callback_df.groupby(['scenario', 'callback']).agg({
            'duration_ms': ['mean', 'std', 'min', 'max', lambda x: x.quantile(0.95)]
        }).round(2)
        cb_stats.columns = ['mean_ms', 'std_ms', 'min_ms', 'max_ms', 'p95_ms']
        cb_stats.to_csv(output_dir / 'callback_statistics.csv')
        print(f"Saved: {output_dir / 'callback_statistics.csv'}")
    
    # Bottlenecks
    if len(bottleneck_df) > 0:
        bottleneck_df.to_csv(output_dir / 'bottlenecks.csv', index=False)
        print(f"Saved: {output_dir / 'bottlenecks.csv'}")
    
    # Generate Mermaid diagram
    mermaid = generate_mermaid_diagram(callback_df)
    with open(output_dir / 'callback_flow.mmd', 'w') as f:
        f.write(mermaid)
    print(f"Saved: {output_dir / 'callback_flow.mmd'}")

def generate_mermaid_diagram(df: pd.DataFrame) -> str:
    """
    Generate a Mermaid flowchart of callback chains.
    """
    lines = ['flowchart TD']
    
    if 'chain' not in df.columns:
        return ''
    
    # Get unique chains and callbacks
    for chain in df['chain'].unique():
        chain_df = df[df['chain'] == chain].drop_duplicates('callback')
        callbacks = chain_df['callback'].tolist()
        
        # Add subgraph
        lines.append(f'    subgraph {chain}')
        
        for i, cb in enumerate(callbacks):
            cb_id = cb.replace('_', '')
            mean_ms = df[df['callback'] == cb]['duration_ms'].mean()
            lines.append(f'        {cb_id}["{cb}<br/>{mean_ms:.1f}ms"]')
            
            if i > 0:
                prev_id = callbacks[i-1].replace('_', '')
                lines.append(f'        {prev_id} --> {cb_id}')
        
        lines.append('    end')
    
    return '\n'.join(lines)

# Uncomment to export
# if len(callback_df) > 0:
#     bottleneck_df = identify_bottlenecks(callback_df) if 'bottleneck_df' not in dir() else bottleneck_df
#     export_path_analysis(callback_df, bottleneck_df, OUTPUT_DIR)

In [ ]:
# Generate summary markdown
if len(callback_df) > 0:
    print("\n" + "="*60)
    print("PATH ANALYSIS SUMMARY")
    print("="*60)
    
    print(f"\nTotal callback records: {len(callback_df)}")
    print(f"Scenarios: {callback_df['scenario'].nunique()}")
    print(f"Unique callbacks: {callback_df['callback'].nunique()}")
    
    if 'chain' in callback_df.columns:
        print(f"Callback chains: {callback_df['chain'].nunique()}")
    
    print("\nTop 5 Slowest Callbacks (mean):")
    top_slow = callback_df.groupby('callback')['duration_ms'].mean().nlargest(5)
    for cb, duration in top_slow.items():
        print(f"  {cb}: {duration:.1f}ms")
    
    if 'bottleneck_df' in dir() and len(bottleneck_df) > 0:
        print(f"\nBottlenecks identified: {len(bottleneck_df)}")
        for _, row in bottleneck_df.iterrows():
            print(f"  {row['scenario']}/{row['callback']}: {row['pct_total']:.1f}%")

## Next Steps

1. Run experiments with tracing enabled:
   ```bash
   cd ~/ldos_manip_tracing
   make run_all NUM_TRIALS=30
   ```

2. Analyze traces with e2e_path_analyzer:
   ```bash
   python3 analysis/e2e_path_analyzer.py --trace-dir traces/baseline/ --output analysis/output/paths/
   ```

3. Review generated reports and Mermaid diagrams